In [1]:
#!/usr/bin/env python

In [2]:
class Sample:
    before = None
    instruction = None
    after = None
    def __init__(self, before, instruction, after):
        self.before = before
        self.instruction = instruction
        self.after = after

In [3]:
class Operator:
    name = "NOP"
    
    @staticmethod
    def execute(registers, instruction):
        return registers

class AddR(Operator):
    name = "ADDR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] + registers[instruction[2]]
        return new_registers

class AddI(Operator):
    name = "ADDI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] + instruction[2]
        return new_registers

class MulR(Operator):
    name = "MULR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] * registers[instruction[2]]
        return new_registers

class MulI(Operator):
    name = "MULI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] * instruction[2]
        return new_registers

class BanR(Operator):
    name = "BANR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] & registers[instruction[2]]
        return new_registers

class BanI(Operator):
    name = "BANI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] & instruction[2]
        return new_registers

class BorR(Operator):
    name = "BORR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] | registers[instruction[2]]
        return new_registers

class BorI(Operator):
    name = "BORI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]] | instruction[2]
        return new_registers

class SetR(Operator):
    name = "SETR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = registers[instruction[1]]
        return new_registers

class SetI(Operator):
    name = "SETI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = instruction[1]
        return new_registers

class GtIR(Operator):
    name = "GTIR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(instruction[1] > registers[instruction[2]])
        return new_registers

class GtRI(Operator):
    name = "GTRI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(registers[instruction[1]] > instruction[2])
        return new_registers

class GtRR(Operator):
    name = "GTRR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(registers[instruction[1]] > registers[instruction[2]])
        return new_registers

class EqIR(Operator):
    name = "EQIR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(instruction[1] == registers[instruction[2]])
        return new_registers

class EqRI(Operator):
    name = "EQRI"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(registers[instruction[1]] == instruction[2])
        return new_registers

class EqRR(Operator):
    name = "EQRR"
    
    @staticmethod
    def execute(registers, instruction):
        new_registers = list(registers)
        new_registers[instruction[3]] = int(registers[instruction[1]] == registers[instruction[2]])
        return new_registers

In [4]:
OPERATORS = [AddI, AddR, MulI, MulR, BanI, BanR, BorI, BorR, SetI, SetR, GtIR, GtRI, GtRR, EqIR, EqRI, EqRR]

In [5]:
def get_input(fname):
    samples = []
    instructions = []
    with open(fname) as f:
        lines = [l.strip() for l in f.readlines()]
    i = 0
    while i < len(lines):
        s = lines[i].split(' ')
        if s[0] == 'Before:':
            before = list(map(int, [s[1].lstrip('[').rstrip(','), s[2].rstrip(','), s[3].rstrip(','), s[4].rstrip(']')]))
            registers = list(map(int, lines[i+1].split(' ')))
            s = lines[i+2].split(' ')[1:]
            after = list(map(int, [s[1].lstrip('[').rstrip(','), s[2].rstrip(','), s[3].rstrip(','), s[4].rstrip(']')]))
            sample = Sample(before, registers, after)
            samples.append(sample)
            i += 4
        else:
            i += 1
            if len(s) == 1:
                continue
            instructions.append(list(map(int, s)))
    return samples, instructions

In [6]:
test_samples, test_instructions = get_input("test.txt")

In [7]:
def match_operators(sample):
    operators = set()
    for op in OPERATORS:
        after = op.execute(sample.before, sample.instruction)
        if after == sample.after:
            operators.add(op)
    return operators

In [8]:
print(len([o for o in [match_operators(s) for s in test_samples] if len(o) >= 3]))

1


In [9]:
samples, instructions = get_input("input.txt")

In [10]:
print(len([o for o in [match_operators(s) for s in samples] if len(o) >= 3]))

596


In [11]:
ops = { i: set(OPERATORS) for i in range(16) }

In [12]:
for s in samples:
    ops[s.instruction[0]] = ops[s.instruction[0]] & match_operators(s)

In [13]:
operators = {}
while len(operators) < len(OPERATORS):
    operation = None
    for idx, op in ops.items():
        if len(op) == 1:
            operation = op.pop()
            operators[idx] = operation
            del ops[idx]
            break
    if operation is not None:
        for op in ops.values():
            op.discard(operation)

In [14]:
registers = [0, 0, 0, 0]
for instruction in instructions:
    registers = operators[instruction[0]].execute(registers, instruction)

In [15]:
print(registers)

[554, 2, 3, 554]
